In [207]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [208]:
import pandas as pd 
import pandas as pd
import sqlite3

conn = sqlite3.connect('database-1.db')


In [209]:
import sqlite3
import os
import pandas as pd

conn = sqlite3.connect('allotrac.db')
cur = conn.cursor()

base_path = "hackathon2024/csv-dataset"

for csvpath in os.listdir(base_path):
  if csvpath.endswith('.csv'):
    # now upload data to sqllite
    table_name = csvpath.split('.')[0]

    df = pd.read_csv(f'{base_path}/{csvpath}')
    df.to_sql(table_name, conn, if_exists='replace', index=False)

In [210]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
  azure_deployment="gpt-4",
  model_name="gpt-4",
  azure_endpoint=os.environ.get("API_ENDPOINT"),
  openai_api_version=os.environ.get("API_VERSION"),
  openai_api_key=os.environ.get("API_KEY"),
  temperature=0
)

In [222]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

connection_uri = f"sqlite:///allotrac.db"

db = SQLDatabase.from_uri(connection_uri, include_tables=list(table_source_map.keys()), sample_rows_in_table_info=4)

db_chain = SQLDatabaseChain.from_llm(llm, db)

db_chain.return_sql = True
db_chain.return_direct = True


In [223]:
from langchain.prompts import PromptTemplate

query_gen_tpl = """You are an expert sql query generator who understands the question in english and convert to sql query strictly based on the context. 
Human: {question}
Assistant: 
"""

In [224]:
def chat(query):
    r = db_chain.run(query_gen_tpl.replace("{question}", query))
    return r

In [234]:
question = "Which truck has the highest capacity? Give me its details"
r2 = chat(question)

In [235]:
print(r2)

SELECT "ID", "Make", "Capacity", "VehicleRegistration" FROM truck ORDER BY "Capacity" DESC LIMIT 1


In [236]:
query_result = db.run(r2)

In [237]:
from langchain.chains import LLMChain

chain_template = """
{history}

based on above history, I have below questions.

Question: {question}

Answer: Explain these result data in human readable format and generate human readable summary. Don't mention the datasource. 
"""

chain_prompt = PromptTemplate.from_template(chain_template)

In [238]:
llm_chain = LLMChain(prompt=chain_prompt, llm=llm)

result = llm_chain.run(
    {
        "history": f"{question} RESULT QUERY IS : \n {r2} \n RETURN FROM DB: \n {query_result}",
        "question": "{question}"
    }
)

print(result)

The truck with the highest capacity is a Mitsubishi with a capacity of 2900.0. Its vehicle registration number is 4-Y9715 and it has an ID of 79.
